In [1]:
%matplotlib inline

import math
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import integrate
from scipy import special
from scipy.integrate import ode


sns.set(style="darkgrid")

# Homework 4
## Problem 1

A d-dimensional hypersphere is defined by the condition:
$$x_{o}^2 + x_{1}^2 + \ldots + x_{d-1}^2 \leq R^2$$
Its volume can be calculated analytically using the formula:
$$V_d (R) = \frac{\pi^{d/2} R^d}{\Gamma(\frac{d}{2} + 1)}$$

Write a program to determine the volume of unit hypershperes (R = 1) using a MonteCarlo integration method, for d = 2,3,4,5,6. You can use either the Hit-or-Miss or the Sample Mean method.

Produce the following using a "good" PRNG, like D1(A1r) or the Mersenne Twister (which one did you use?):